## In this notebook, we will see how we can use ``CheckProject`` with different options.

#### Let's first see the available arguments in ``CheckProject``

- ``config``: Path to the config file or the config itself.
- ``on_dbfs``: Whether you are running on Databricks or not. Default: ``True``.
- ``write_results``: Whether you want to write results to a Delta table or not. Default: ``True``.
- ``dataframe``: Dataframe object. Default: ``None``.
- ``foreign_dataframe``: Dataframe object. Default: ``None``.

#### Import necessary modules and set up necessary things

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from dqf.main import CheckProject


spark = SparkSession.builder.getOrCreate()

config = "path/to/the/config/file.json"

### 1. Using default options

In [ ]:
check_project = CheckProject(config)
check_project.run()

### 2. Specifying that you are not on Databricks

In [ ]:
check_project = CheckProject(config, on_dbfs=False)
check_project.run()

### 3. Specifying that you do not want to write the results to a Delta table

In [ ]:
check_project = CheckProject(config, write_results=False)
check_project.run()

### 4. Suppose you have filtered your dataframe and now want to use this dataframe but with the same config, you can specify that in the ``dataframe`` argument.

In [ ]:
dataframe = spark.read.parquet("/mnt/cocbigdatard-prod/projects/DataQualityFramework/Testing/data/input/cars_Database_All_Formats/parquet/customer_cars_dataset_ok/cars")

filter_dataframe = dataframe.filter(F.col("car_id") > 10)


check_project = CheckProject(config, dataframe=filter_dataframe)
check_project.run()

### 5. Suppose you have filtered two dataframes (one of them is a foreign dataframe) and now want to use these dataframes but with the same config, you can specify that in the ``dataframe`` and ``foreign_dataframe`` arguments. This is only for [check_foreign_key](https://git.daimler.com/cocbigdatard/DataQualityFramework/tree/development/docs/dqf_quality_checks.md#check_foreign_key) check.

In [ ]:
cars = spark.read.parquet("/mnt/cocbigdatard-prod/projects/DataQualityFramework/Testing/data/input/cars_Database_All_Formats/parquet/customer_cars_dataset_ok/cars")
customers = spark.read.parquet("/mnt/cocbigdatard-prod/projects/DataQualityFramework/Testing/data/input/cars_Database_All_Formats/parquet/customer_cars_dataset_ok/customers")

filter_cars = cars.filter(F.col("car_id") > 10)
filter_customers = customers.filter(F.col("customer_id") > 10)


check_project = CheckProject(config, dataframe=filter_cars, foreign_dataframe=filter_customers)
check_project.run()